# 역전파 (Backpropagation)
### 연쇄법칙
 - 기본 수식의 역전파 & 연쇄법칙 적용

In [3]:
import numpy as np

def forward(x):
    y = x**2
    return y

def backward(x):
    dy_dx = 2*x
    return dy_dx

x = 3.0

print(forward(x))
print(backward(x))


9.0
6.0


- 다층 신경망에서 연쇄법칙 적용

In [4]:
import numpy as np

def forward(x):
    y = x ** 2  # 예제 함수: y = x^2
    z = 2 * y   # z = 2y
    return z

def backward(x):
    dy_dx = 2 * x  # y = x^2의 미분
    dz_dy = 2      # z = 2y의 미분
    dz_dx = dz_dy * dy_dx  # 연쇄법칙 적용
    return dz_dx

x = 3.0
print("순전파 결과:", forward(x))
print("역전파 결과 (기울기):", backward(x))

순전파 결과: 18.0
역전파 결과 (기울기): 12.0


### 신경망에서의 활용

 - 단순 신경망 학습

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x) * (1-sigmoid(x))

x = np.array([0.5, 0.8])
y = np.array([1])

w = np.array([0.2, 0.4])

# 순전파
z = np.dot(x, w)
r = sigmoid(z)

# 오차 계산
loss = 0.5 * (y - r)**2

# 역전파 (기울기 계산)
delta = (y - r)*sigmoid_d(z)
grad_w = delta * x

# 가중치 갱신
w -= 0.1 * grad_w

print(w)

[0.19525585 0.39240936]


- 은닉층 추가

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_d(x):
    return np.where(x > 0, 1, 0)

x = np.array([0.5, 0.8]) # (2, )
y = np.array([1])

w1 = np.array([[0.2, 0.4], [0.1, 0.3]]) # (2, 2)
b1 = np.array([0.1, 0.2]) #(2, )
w2 = np.array([[0.5], [0.6]]) # (2, 1)
b2 = np.array([0.3])

# 순전파
z1 = np.dot(x, w1) + b1
r1 = sigmoid(z1)

z2 = np.dot(r1, w2) + b2
r2 = relu(z2)

# 오차 계산
loss = 0.5 * (y - r)**2

# 역전파 (기울기 계산)
# 얻을 수 있는 insight 3가지
# 1) 출력값이 아닌 오차값을 가지고 역전파를 한다는 것, 2) 내 input에 대해서만 사용할 수 있는 r1과 x로 국소적 계산을 하는 것, 3) 이전 층에서 전파해준 기울기를 전파해서 사용해주는 것
delta2 = (r2 - y) * relu_d(z2)      # insight1: r2 - y: 오차값(예측값 - 실제값) 즉, 손실(오차값)과 활성화함수를 미분한 것(relu_d)을 가지고 오차를 구함
grad_w2 = np.outer(r1, delta2)      # insight2: 나(r1)에 대한 input을 사용하는 것

delta1 = np.dot(w2, delta2) * relu_d(z1)        # insight3: 이전 층에서 전파해준 기울기를 적용하는 것 # 내적
grad_w1 = np.outer(x, delta1)                   # 외적

# 가중치 갱신
# 손실이 가장 최소가 되는 지점을 향해 가야하니까 경사하강법을 통해 손실 최소화 지점을 찾아가는 것임
learning_rate = 0.01
w2 -= learning_rate * grad_w2
w1 -= learning_rate * grad_w1

print(w1)
print(w2)

[[0.20005594 0.40006712]
 [0.1000895  0.3001074 ]]
[[0.50012743]
 [0.6001465 ]]


---

### 수치미분과 역전파

In [7]:
def f(x):
    return x**2

# 수치미분: 미분을 근사하는 방법
# 아주 작은 값을 넣어 중앙차분
def num_d_gradient(f, x):
    h = 1e-5
    return (f(x + h) - f(x - h)) / (2 * h)      # 중앙차분

# 미분
# 역전파는 결국 기울기를 구하는거라 미분이 적용됨
def backward_gradient(x):
    return 2 * x

# num_d_gradient보다 backward_gradient가 더 정확한 값이 나온 이유는 계산 자체가 미분이 더 딱 떨어지게 만들었기 때문
# 역전파법에서는 그냥 미분을 사용할 것임
print(num_d_gradient(f, 3.0))   # 수치미분 결과
print(backward_gradient(3.0))   # 오차역전파법 결과

6.000000000039306
6.0


---

##### 숫자 맞추기 AI

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x) * (1-sigmoid(x))

target_number = 27

guess = np.random.randn()   # 임의의 값을 예측 숫자로 잡기 # 가우시안 분포

learning_rate = 0.1

for i in range(100):         # 반복 횟수는 epoch이 됨

    # 오차 계산
    loss = 0.5 * (guess - target_number)**2

    # 역전파 (기울기 계산) 오차에 대한 변화량만 가지고 sigmoid를 사용하지 않을거임
    grad = (guess - target_number) # 오차에 대한 변화량 # 원래식: 1/2 * (실제값 - 예측값)**2 -> 미분하면 (예측값 - 실제값) 이라 이렇게 식이 나옴

    # 업데이트 (guess 업데이트)
    guess -= learning_rate * grad

    # epoch 5마다 예측값과 손실 출력
    if i%10 == 0:
        print(f'epoch {i} | 예측값: {guess}, 손실: {loss}')

# 최종 예측값 guess 출력
print(f'최종 예측값: {guess}')

epoch 0 | 예측값: 2.915970562834806, 손실: 358.04967526558
epoch 10 | 예측값: 18.602418184526762, 손실: 43.53048169602889
epoch 20 | 예측값: 24.071944271968665, 손실: 5.292290337319207
epoch 30 | 예측값: 25.979050096224164, 손실: 0.6434189543332641
epoch 40 | 예측값: 26.644016780131196, 손실: 0.0782247239680004
epoch 50 | 예측값: 26.875876326194373, 손실: 0.009510300246300072
epoch 60 | 예측값: 26.95672075103797, 손실: 0.0011562304880970355
epoch 70 | 예측값: 26.98490945898322, 손실: 0.00014057063467845234
epoch 80 | 예측값: 26.994738253698003, 손실: 1.7090107497886353e-05
epoch 90 | 예측값: 26.998165342507217, 손실: 2.0777580961855314e-06
최종 예측값: 26.999289216096997
